In [1]:
from utils.EWMA import EWMA
from utils.RSI import RSI
from utils.SMA import SMA
from utils.ADX import ADX
from utils.SO import STO
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import optuna 

/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_5m = pd.read_csv('data/aapl_1h_test.csv')


In [3]:
# Parámetros predefinidos
params = {
    'stop_loss_long': 0.024796600103236185,
    'take_profit_long': 0.38593322012654585,
    'stop_loss_short': 0.49615439857382665,
    'take_profit_short': 0.6340815141615455,
    'n_shares': 95,
    'sma_long_window': 24,
    'sma_short_window': 14,
    'sto_window': 10,
    'sto_upper_threshold': 81,
    'sto_lower_threshold': 19
}

# Variables iniciales
cash = 1_000_000
active_operations = []
com = 0.00125  # Comisión
strategy_value = [1_000_000]

In [10]:
def run_strategies(df, params, cash, active_operations, com, strategy_value):
    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    # Ejecución de SMA
    sma_buy_signals, sma_sell_signals, sma_strategy_value = SMA(
        df, cash, active_operations, com, strategy_value,
        params['sma_long_window'], params['sma_short_window'],
        params['stop_loss_long'], params['take_profit_long'],
        params['stop_loss_short'], params['take_profit_short'], params['n_shares'], fillna=0
    )
    buy_signals["SMA"] = sma_buy_signals
    sell_signals["SMA"] = sma_sell_signals

    # Ejecución de STO
    sto_buy_signals, sto_sell_signals, sto_strategy_value = STO(
        df, cash, active_operations, com, strategy_value,
        params['sto_window'], params['sto_upper_threshold'], params['sto_lower_threshold'],
        params['stop_loss_long'], params['take_profit_long'],
        params['stop_loss_short'], params['take_profit_short'], params['n_shares'], fillna=0
    )
    buy_signals["STO"] = sto_buy_signals
    sell_signals["STO"] = sto_sell_signals

    # Combine sma_strategy_value and sto_strategy_value
    strategy_value = [sma + sto for sma, sto in zip(sma_strategy_value, sto_strategy_value)]

    for i, row in buy_signals.iterrows():
        # If all indicators want to buy...
        if sum(buy_signals.iloc[i,:]) == len(buy_signals.columns):
            buy_signal = True
        else:
            buy_signal = False
        buy_signals.loc[i, "General"] = buy_signal

    for i, row in sell_signals.iterrows():
        # If all indicators want to sell...
        if sum(sell_signals.iloc[i,:]) == len(sell_signals.columns):
            sell_signal = True
        else:
            sell_signal = False
        sell_signals.loc[i, "General"] = sell_signal

    return buy_signals, sell_signals, strategy_value

In [11]:
buy_signals, sell_signals, strategy_value = run_strategies(df_5m, params, cash, active_operations, com, strategy_value)


TypeError: SMA.__init__() got an unexpected keyword argument 'fillna'